In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Input,Flatten,Reshape,AveragePooling2D,Dropout,LayerNormalization, ReLU,concatenate,Cropping2D, BatchNormalization
from keras.datasets import mnist
import tensorflow as tf

import keras
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
image_path = '../dados/CAPTCHA-10k/teste'
def generate_df(image_path):
  label_path = '../dados/CAPTCHA-10k/labels10k'

  jpg_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
  jpg_files.sort()
  data = []

  for jpg_file in jpg_files:
      txt_file = os.path.splitext(jpg_file)[0] + '.txt'
      txt_file_path = os.path.join(label_path, txt_file)

      if os.path.exists(txt_file_path):
          with open(txt_file_path, 'r') as file:
              txt_content = file.read().strip()

          data.append({'jpg_file': jpg_file, 'txt_content': txt_content})
  return pd.DataFrame(data)

df = generate_df(image_path)
df.head()

In [ ]:
vocab = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B',
       'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
       'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
np.array(vocab)

In [ ]:
def preprocess(img):
  kernel  = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
  img     = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
  _, img  = cv2.threshold(img, 90, 255, cv2.THRESH_BINARY)
  return img

In [ ]:
def generate_X_Y(image_path):
  df = generate_df(image_path)
  X = [preprocess(cv2.imread(os.path.join(image_path, x),cv2.IMREAD_GRAYSCALE)) for x in df["jpg_file"]]
  X = np.array(X)
  X = np.expand_dims(X, axis=-1)

  X = X.astype('float32') / 255.
  return X,df['txt_content']

X_teste,labels_teste = generate_X_Y('../dados/CAPTCHA-10k/teste')

In [ ]:
plt.imshow(X_teste[0], cmap='gray')

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def psnr(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)

In [ ]:
autoencoder = tf.keras.models.load_model('model_MSE_aug_best_unet.tf',custom_objects={"rmse": rmse,"psnr":psnr})
classifier = tf.keras.models.load_model('classifier_pre_trained.tf')

In [ ]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam()

# Setup checkpoint and checkpoint manager
checkpoint_dir = './checkpoints'
checkpoint = tf.train.Checkpoint(autoencoder=autoencoder, classifier=classifier, optimizer=optimizer)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)

# Restore the latest checkpoint
checkpoint.restore(checkpoint_manager.latest_checkpoint)

if checkpoint_manager.latest_checkpoint:
    print(f"Restored from {checkpoint_manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")

In [ ]:
plt.imshow(autoencoder(X_teste[:1])[0],cmap="grey")

In [ ]:
def inference(input_images):
    # Forward pass through the autoencoder
    autoencoder_output = autoencoder.predict(input_images)
    predictions_list = []

    interval = [0, 30, 60, 90, 120, 150, 180]

    for i in range(len(interval) - 1):
        fake_img = autoencoder_output[:, :, interval[i]:interval[i + 1], :]
        y_pred = classifier(fake_img, training=False)
        predictions_list.append(y_pred)

    # Concatenate predictions from all patches
    predictions = tf.stack(predictions_list,axis=1)
    predictions = tf.argmax(predictions, axis=2)  # Convert to class indices
    return predictions.numpy()

preds = inference(X_teste)

In [ ]:
vocab_tensor = np.array(vocab)

In [ ]:
all_preds = vocab_tensor[preds]
words = ["".join(list(word)) for word in all_preds]
df["preds"] = words

In [ ]:
df.head()

In [ ]:
def count_matches(row):
    txt_content = row['txt_content']
    pred = row['preds']
    return sum(1 for a, b in zip(txt_content, pred) if a == b)

# Apply the function to each row
df['matches'] = df.apply(count_matches, axis=1)
df.sort_values(by="matches").head(20)

In [ ]:
match_counts = df['matches'].value_counts().sort_index()

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.bar(match_counts.index, match_counts.values, color='skyblue')
plt.xlabel('Number of Matches')
plt.ylabel('Count')
plt.title('Count of Matches per Row')
plt.xticks(match_counts.index)  # Ensure x-axis labels match the match counts
plt.grid(axis='y', linestyle='--')

# Show the plot
plt.show()

In [ ]:
miss = (df["matches"] != df["txt_content"].str.len()).sum()
acc = (len(df) - miss)/len(df)
acc

In [ ]:
# Calculate recognition rates
max_chars = df['txt_content'].map(len).max()
recognition_rates = []

for min_correct in range(1, max_chars + 1):
    count_correct = df['matches'] >= min_correct
    rate = count_correct.mean()
    recognition_rates.append(rate)

# Plot the recognition rates
plt.figure(figsize=(10, 6))
plt.axhline(y=acc, color='r', linestyle='--', label=f'Overall Accuracy: {acc:.2f}')
plt.plot(range(1, max_chars + 1), recognition_rates, marker='o', color='skyblue', label='Recognition Rate')
plt.xlabel('Minimum Number of Correct Characters')
plt.ylabel('Recognition Rate')
plt.title('Recognition Rate vs Minimum Number of Correct Characters')
plt.grid(axis='y', linestyle='--')
plt.ylim(0, 1)
plt.xticks(range(1, max_chars + 1))
plt.legend()
plt.show()